In [31]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder

pd.set_option("display.max_columns", None)

In [32]:
bank = pd.read_csv("../data/Casestudy Data.csv")

In [33]:
bank.shape

(41188, 21)

In [34]:
bank.head

<bound method NDFrame.head of         age          job  marital            education  default housing loan  \
0      56.0    housemaid  married             basic.4y       no      no   no   
1      57.0     services  married          high.school  unknown      no   no   
2      37.0     services  married          high.school       no     yes   no   
3      40.0       admin.  married             basic.6y       no      no   no   
4      56.0     services  married          high.school       no      no  yes   
...     ...          ...      ...                  ...      ...     ...  ...   
41183  73.0      retired  married  professional.course       no     yes   no   
41184  46.0  blue-collar  married  professional.course       no      no   no   
41185  56.0      retired  married    university.degree       no     yes   no   
41186  44.0   technician  married  professional.course       no      no   no   
41187  74.0      retired  married  professional.course       no     yes   no   

         

In [35]:
bank.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,37444.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,36040.000000,41188.000000,41188.000000,41188.000000
mean,39.999947,258.285010,2.567593,962.475454,0.172963,0.081886,93.575575,-40.502600,3.621291,5167.035911
std,10.414908,259.279249,2.770014,186.910907,0.494901,1.570960,0.579216,4.628198,1.734447,72.251528
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [36]:
bank.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             37444 non-null  float64
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  36040 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [37]:
bank = bank.replace("unknown", value = np.nan)
bank = bank.replace("yes", value = 1)
bank = bank.replace("no", value = 0)
bank = bank.replace("success", value = 1)
bank = bank.replace("failure", value = 0)
bank = bank.replace("nonexistent", value = np.nan)

In [38]:
bank.describe()

,age,default,housing,loan,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
count,37444.000000,32591.000000,40198.000000,40198.000000,41188.000000,41188.000000,41188.000000,41188.000000,5625.000000,41188.000000,36040.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,39.999947,0.000092,0.536743,0.155431,258.285010,2.567593,962.475454,0.172963,0.244089,0.081886,93.575575,-40.502600,3.621291,5167.035911,0.112654
std,10.414908,0.009594,0.498654,0.362319,259.279249,2.770014,186.910907,0.494901,0.429584,1.570960,0.579216,4.628198,1.734447,72.251528,0.316173
min,17.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000,0.000000
25%,32.000000,0.000000,0.000000,0.000000,102.000000,1.000000,999.000000,0.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000,0.000000
50%,38.000000,0.000000,1.000000,0.000000,180.000000,2.000000,999.000000,0.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000,0.000000
75%,47.000000,0.000000,1.000000,0.000000,319.000000,3.000000,999.000000,0.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000,0.000000
max,98.000000,1.000000,1.000000,1.000000,4918.000000,56.000000,999.000000,7.000000,1.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000,1.000000


In [39]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             37444 non-null  float64
 1   job             40858 non-null  object 
 2   marital         41108 non-null  object 
 3   education       39457 non-null  object 
 4   default         32591 non-null  float64
 5   housing         40198 non-null  float64
 6   loan            40198 non-null  float64
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        5625 non-null   float64
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  36040 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [40]:
# drop "duration" column based on the description
bank = bank.drop(columns = ["duration"])

bank_X = bank.drop(columns = ["y"])
bank_y = bank["y"]
cate_features = []

for col in bank_X.columns:
    if bank_X[col].dtype == "object":
        bank_X[col] = bank_X[col].astype("category")
        cate_features.append(col)


In [41]:
# Show "N/A" counts
np.sum(bank_X.isna())

age                3744
job                 330
marital              80
education          1731
default            8597
housing             990
loan                990
contact               0
month                 0
day_of_week           0
campaign              0
pdays                 0
previous              0
poutcome          35563
emp.var.rate          0
cons.price.idx     5148
cons.conf.idx         0
euribor3m             0
nr.employed           0
dtype: int64

In [42]:
# Rescale "pdays"
bank_X["pdays"] = bank_X["pdays"] / np.sqrt(np.var(bank_X["pdays"]))

bank_X.join(bank_y).to_csv("../data/bank.csv", index = False)

In [43]:
# Generating dummy variables
bank_X_flattened = pd.get_dummies(bank_X, dummy_na = False)
bank_X_flattened.join(bank_y).to_csv("../data/bank_flattened.csv", index = False)

In [44]:
# Iterative imputation
imp = IterativeImputer()
bank_X_imputed_imp = pd.DataFrame(imp.fit_transform(bank_X_flattened), columns = bank_X_flattened.columns)
bank_X_imputed_imp.join(bank_y).to_csv("../data/bank_imputed_imp.csv", index = False)
bank_X_imputed_imp.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            41188 non-null  float64
 1   default                        41188 non-null  float64
 2   housing                        41188 non-null  float64
 3   loan                           41188 non-null  float64
 4   campaign                       41188 non-null  float64
 5   pdays                          41188 non-null  float64
 6   previous                       41188 non-null  float64
 7   poutcome                       41188 non-null  float64
 8   emp.var.rate                   41188 non-null  float64
 9   cons.price.idx                 41188 non-null  float64
 10  cons.conf.idx                  41188 non-null  float64
 11  euribor3m                      41188 non-null  float64
 12  nr.employed                    41188 non-null 